## Scrape KHMR

Driver for scraping data from Kicking Horse Mountain Resort in Golden, BC, Canada. Scrapes data from the URL:

http://kickinghorseresort.com/conditions/advanced-weather-data/

and outputs weather data for weewx processing and display etc

In [43]:
import pandas as pd
import requests
import bs4
import datetime
import time

In [7]:
# scrape this URL
the_url = 'http://kickinghorseresort.com/conditions/advanced-weather-data/'

In [15]:
# parse the page with requests + BeautifulSoup
the_page = requests.get(the_url)
the_soup = bs4.BeautifulSoup(the_page.content, 'html.parser').findAll()
the_split = str(the_soup).split("\r\n")

In [14]:
the_str[7:]

['}</style></meta></meta></head>\n<body class="page-template-default page page-id-16615 page-child parent-pageid-16176 res1170 kl-follow-menu kl-skin--light" itemscope="itemscope" itemtype="https://schema.org/WebPage">\n<div class="msgmodal" id="msgModal"><div class="msgmodal-content"><span id="closemodal" onclick="closemodalwindow();">×</span><div id="msgText"></div></div></div>\n<div class="backdrop" id="mobilebackdrop" onclick="javascript:closePanel(\'All\');"></div>\n<div class="panelwrapperdesktop" id="panelSnowReportTop">\n<div class="panelsnowreport">\n<div class="rcrtabcontent" id="weather">\n<div class="weatherwrapper">\n<a class="panelclosebtn" href="javascript:void(0)" onclick="closePanel(\'All\')">×</a>\n<div class="panelsnowreport">\n<div style="width:100%;max-width:1200px;margin: 0 auto;">\n<div id="header-container-snowreport">\n<div class="header-snow-title"><a href="/conditions/snow-report/">SNOW CONDITIONS » </a></div>\n<table class="table-snowreport-header">\n<tr><td

In [38]:
# figure out what day it is to search just the data we want
today_day = datetime.datetime.today().day
today_month = datetime.datetime.today().month
today_year = datetime.datetime.today().year

my_lines = []
for line in the_split:
    if line.__contains__(f"   {today_month} {today_day}    "):
        print(line)
        my_lines.append(line)


             10 29       0    -5.7     133      36     208      49    12.5
             10 29     100    -5.7     168      35     202      44    12.5
             10 29     200    -5.6     199      31     200      52    12.5
             10 29     300    -5.6     229      30     209      44    12.5
             10 29     400    -5.6     259      30     212      39    12.4
             10 29     500    -5.7     287      28     214      39    12.4
             10 29     600    -6.0     317      30     209      43    12.4
             10 29     700    -6.3     351      34     206      44    12.4
             10 29     800    -6.3     378      27     212      42    12.3
             10 29     900    -6.5     407      29     207      45    12.5
             10 29    1000    -6.4     442      35     208      51    13.1
             10 29    1100    -5.8     476      34     208      47    16.6
             10 29    1200    -5.3     503      27     207      36    16.6
             10 29    130

In [30]:
my_line = my_lines[6]

In [50]:
my_data = [item for item in my_line.split(" ") if item != ""]

In [51]:
my_data

['10', '29', '600', '-6.0', '317', '30', '209', '43', '12.4']

In [54]:
my_date = datetime.datetime(year=today_year, month=int(my_data[0]), day=int(my_data[1]), hour=int(my_data[2][:-2]))
my_date

datetime.datetime(2022, 10, 29, 6, 0)

In [46]:
time.mktime(my_date.timetuple())

1665730800.0

In [56]:
packet = {}
packet["dateTime"] = time.mktime(my_date.timetuple())

In [57]:
packet

{'dateTime': 1667048400.0}